In [84]:
import pandas as pd
import re
import os
import pickle


TODO

    Procent odpowiedzi poprawnych (accuracy) w zależności od parametru lambda.
    Liczba różnych odpowiedzi (tzn. odp_uni != odp_bigram) w zależności od parametru lambda.
    Tabele poprawności według kategorii dla każdej z wartości parametru lambda.



# TODO
1. Procent poprawnych (accuracy, czyli skuteczność) w zależności od parametru lambda. DONE

2. Liczba różnych odpowiedzi (tzn. odp_uni != odp_bigram) w zależności od parametru lambda. DONE

3. Skuteczność z podziałem na: type, category, basic_type_operation w zależności od parametru lambda. Done

4. Funkcja generująca tabele ze skutecznością powinna:
    a. sprawdzić czy `None` występują zawsze parami tzn. w modelu uni- i bigramowym - inaczej error DONE
    
    b. wyniki wspólne dla wszystkich zapisywać do oddzielnej ramki danych, tzn. % None, skutecznośc uni, ogólna (bez podziału) skuteczność bi
    c. funkcja nie może sortować ramek pośrednich po accuracy, bo wtedy się nie połączą w jedną dużą DONE
    
    d. dla każdej lambdy tworzony jest słownik filtrowanych ramek danych DONE
    
    e. oddzielna ramka danych ze zliczeniem: wg type, wg category, wg basic_operation_type, wg type+basic_op, wg category+basic_op, wg type+cat i wg wszystkich trzech DONE

Predykcji zmiennej celu, czyli poprawnego zapisu błędu, dokonano przy użyciu następujących wartości parametru `lambda`. Parametr ten jest istotny jedynie z punktu widzenia modelu dwugramowego.

In [2]:
! ls ../results/ | grep test_set

test_set_with_answers_no_lang_error_scaling_0.20_lambda.csv
test_set_with_answers_no_lang_error_scaling_0.33_lambda.csv
test_set_with_answers_no_lang_error_scaling_0.5_lambda.csv
test_set_with_answers_no_lang_error_scaling_0.66_lambda.csv
test_set_with_answers_no_lang_error_scaling_0.75_lambda.csv
test_set_with_answers_no_lang_error_scaling_0.90_lambda.csv
test_set_with_answers_no_lang_error_scaling_0.95_lambda.csv
test_set_with_answers_no_lang_error_scaling_1.0_lambda.csv


W plikach z predykcjami znajdują się także predykcje dla modelu 1-gramowego. Są zawsze takie same, ponieważ model 1-gramowy nie zależy od parametru `lambda`.

Gdy w kolumnach `unigram_case` i/lub `bigram_case` znajduje się `NaN` oznacza to, że model kanału z błędem zwrócił dla analizowanej operacji `None`. `None` wiążę się z: 
 1. brakiem kandydatów (wygenerowane przez algorytm DL słowa nie znajdują się na liście istniejących słów języka polskiego) 
 2. błędem w zbiorze testowym określono ciąg jedynie zakazanych znaków (tzn. innych niż litery alfabetu łacińskiego rozszerzonego o polskie znaki diakrytyczne) - po przeprowadzeniu czyszczenia powstaje pusty napis, z którego nie można generować sensownych nowych słów
 3. wskazany błąd nie znajduje się w zdaniu
 
Przypadki 2 oraz 3 są rzadkie - stanowią nie więcej niż 30 obserwacji.

In [119]:
def load_dataframe(file):
    return pd.read_csv('../results/{}'.format(file))

In [120]:
def get_lambda_from_df(file):
    return re.search('\d\\.\d+', file).group().replace('.', ',')

In [135]:
def check_nones(dataframe):
    """
    Return number of Nones. If number of Nones is not equal among uni- and bigram model raise an error.
    """
    df_unigram_nan = dataframe.loc[dataframe['unigram_case'].isna()]
    df_bigram_nan = dataframe.loc[dataframe['bigram_case'].isna()]
    
    diff_uni_bi = set(df_unigram_nan.index) - set(df_bigram_nan.index)
    diff_bi_uni = set(df_bigram_nan.index) - set(df_unigram_nan.index)
    # empty set casts to False
    if not diff_uni_bi and not diff_bi_uni:
        return df_unigram_nan.shape[0] / dataframe.shape[0]
    else:
        print('Different number of Nones in uni- and bigram model.') 
        print("Uni - bi: ", len(diff_uni_bi))
        print("Bi - uni: ", len(diff_bi_uni)) 
        return df_bigram_nan.shape[0] / dataframe.shape[0]

In [122]:
def get_dataframes_with_answers(dataframe):
    return dataframe.dropna(subset=['unigram_case', 'bigram_case'])

In [123]:
def attach_correctness_column(dataframe):
    """
    Accepts a dataframe that hasn't got any Nones as answers
    """
    
    dataframe['is_correct_uni'] = dataframe.apply(lambda x: int(x['gold_standard'] == x['unigram_case']), axis=1)
    dataframe['is_correct_bi'] = dataframe.apply(lambda x: int(x['gold_standard'] == x['bigram_case']), axis=1)
    
    return dataframe

In [124]:
def get_stats(dataframe):
    """
    Accept a dataframe that hasn't got any Nones as answers and has 0/1 columns informing of answers correctness
    """
    
    stats_dict = dict()
    
    uni_bi_diff_no = dataframe.loc[dataframe['unigram_case'] != dataframe['bigram_case']].shape[0]
    uni_better_bi_no = dataframe.loc[(dataframe['is_correct_uni'] == 1) & (dataframe['is_correct_bi'] == 0)].shape[0]
    bi_better_uni_no = dataframe.loc[(dataframe['is_correct_uni'] == 0) & (dataframe['is_correct_bi'] == 1)].shape[0]
    
    df_correct_uni = dataframe.loc[dataframe['gold_standard'] == dataframe['unigram_case']]
    df_correct_bi = dataframe.loc[dataframe['gold_standard'] == dataframe['bigram_case']]
    
    general_accuracy_uni = df_correct_uni.shape[0] / dataframe.shape[0]
    general_accuracy_bi = df_correct_bi.shape[0] / dataframe.shape[0]
    
    dataframe_full = attach_correctness_column(dataframe)
    
    accuracy_by_type_uni = dataframe_full.groupby(['type']).agg({'is_correct_uni': ['mean']})
    accuracy_by_category_uni = dataframe_full.groupby(['category']).agg({'is_correct_uni': ['mean']})
    accuracy_by_operation_uni = dataframe_full.groupby(['basic_type_operation']).agg({'is_correct_uni': ['mean']})
    accuracy_by_all_uni = dataframe_full.groupby(['type', 'category', 'basic_type_operation']).agg({'is_correct_uni': ['mean']})
    
    accuracy_by_type_bi = dataframe_full.groupby(['type']).agg({'is_correct_bi': ['mean']})
    accuracy_by_category_bi = dataframe_full.groupby(['category']).agg({'is_correct_bi': ['mean']})
    accuracy_by_operation_bi = dataframe_full.groupby(['basic_type_operation']).agg({'is_correct_bi': ['mean']})
    accuracy_by_all_bi = dataframe_full.groupby(['type', 'category', 'basic_type_operation']).agg({'is_correct_bi': ['mean']})
    
    stats_dict['accuracy_by_type_uni'] = accuracy_by_type_uni
    stats_dict['accuracy_by_category_uni'] = accuracy_by_category_uni
    stats_dict['accuracy_by_operation_uni'] = accuracy_by_operation_uni
    stats_dict['accuracy_by_all_uni'] = accuracy_by_all_uni
    
    stats_dict['accuracy_by_type_bi'] = accuracy_by_type_bi
    stats_dict['accuracy_by_category_bi'] = accuracy_by_category_bi
    stats_dict['accuracy_by_operation_bi'] = accuracy_by_operation_bi
    stats_dict['accuracy_by_all_bi'] = accuracy_by_all_bi
    
    stats_dict['general_accuracy_uni'] = general_accuracy_uni
    stats_dict['general_accuracy_bi'] = general_accuracy_bi
    
    stats_dict['uni_bi_diff_no'] = uni_bi_diff_no
    stats_dict['uni_better_bi_no'] = uni_better_bi_no
    stats_dict['bi_better_uni_no'] = bi_better_uni_no
    
    return stats_dict

In [125]:
def get_filtered_dataframes(accuracy_by_all, df_answers):
    """
    Accept a dataframe that contains summary of accuracy for each type, category and operation.
    Based on the aforementioned df filter the dataframe with answers.
    Return a dictionary of filtered dataframes.
    """
    
    dfs_dict = dict()
    
    for i in range(accuracy_by_all.shape[0]):
    
        # key indicates type, category and basic_type_operation
        col_tuple = accuracy_by_all.iloc[i].name
        key = re.sub("/| ", "_", " ".join(col_tuple))

        # value is a dataframe filtered with key elements
        mask = (df_answers['type'] == col_tuple[0]) & (df_answers['category'] == col_tuple[1]) & (df_answers['basic_type_operation'] == col_tuple[2])

        # get percentage of correct answers for each category
        percentage = accuracy_by_all.iloc[i].values[0]

        dfs_dict[key] = (df_answers[mask], percentage)
    
    return dfs_dict

In [126]:
def get_counts_of_observations(dataframe):
    """
    Accepts a dataframe that hasn't got any Nones as answers.
    """
    
    count_by_type = dataframe.groupby(['type']).agg(['count'])['text_with_error'].sort_values(by='count', ascending=False)
    count_by_category = dataframe.groupby(['category']).agg(['count'])['text_with_error'].sort_values(by='count', ascending=False)
    count_by_operation = dataframe.groupby(['basic_type_operation']).agg(['count'])['text_with_error'].sort_values(by='count', ascending=False)
    
    count_by_type_category = dataframe.groupby(['type', 'category']).agg(['count'])['text_with_error'].sort_values(by='count', ascending=False)
    count_by_type_operation = dataframe.groupby(['type', 'basic_type_operation']).agg(['count'])['text_with_error'].sort_values(by='count', ascending=False)
    count_by_all = dataframe.groupby(['type', 'category', 'basic_type_operation']).agg(['count'])['text_with_error'].sort_values(by='count', ascending=False)
    
    with pd.ExcelWriter('../results/count_info/count_excel.xlsx') as writer:
        count_by_type.to_excel(writer, sheet_name='type')
        count_by_category.to_excel(writer, sheet_name='category')
        count_by_operation.to_excel(writer, sheet_name='operation')
        count_by_type_category.to_excel(writer, sheet_name='type_category')
        count_by_type_operation.to_excel(writer, sheet_name='type_operation')
        count_by_all.to_excel(writer, sheet_name='all')
    
    print("Done!")

In [136]:
def wrapper_stats():
    
    # structures for holding data #
    # lambda-accuracy 
    lambda_col = list()
    accuracy_col = list()
    
    # different answers no
    diff_no_col = list()
    uni_better_bi = list()
    bi_better_uni = list()
    
    # single values
    single_values_dict = dict()
    
    # accuracy by type (index-only dataframes)
    
    ##############################
    
    first_pass_complete = False
    
    for filename in sorted(os.listdir('../results/')):
        if filename.endswith('.csv'):
            print("Analysing ", filename)
            print()
            # get dataframe with answers (no Nones)
            df_raw = load_dataframe(filename)
            
            try:
                lambda_par = get_lambda_from_df(filename)
            except:
                print(filename)
            
            # check if Nones correspond to oen another in unigram and bigram
            single_values_dict['pct_nones'] = check_nones(df_raw)
            
            # dataframes that has no Nones
            df_answers = get_dataframes_with_answers(df_raw)
            
            # attach correctness info
            df_answers = attach_correctness_column(df_answers)
            
            # compute stats and keep in a dictionary
            stats_dict = get_stats(df_answers)
            
            if not first_pass_complete:
                # used to gather data about unigram model
                # and initialise accuracy by type/category/operation dataframes
                first_pass_complete = True
                
                # counts must be computed only once
                get_counts_of_observations(df_answers)
                
                # accuracy of unigram model is constant w.r.t. lambda parameter
                single_values_dict['unigram_acc'] = stats_dict['general_accuracy_uni']
                
                # initialisation
                df_acc_type = pd.DataFrame(index=stats_dict['accuracy_by_type_uni'].index)
                df_acc_type['unigram'] = stats_dict['accuracy_by_type_uni'][('is_correct_uni', 'mean')]
                
                df_acc_category = pd.DataFrame(index=stats_dict['accuracy_by_category_uni'].index)
                df_acc_category['unigram'] = stats_dict['accuracy_by_category_uni'][('is_correct_uni', 'mean')]
                
                df_acc_operation = pd.DataFrame(index=stats_dict['accuracy_by_operation_uni'].index)
                df_acc_operation['unigram'] = stats_dict['accuracy_by_operation_uni'][('is_correct_uni', 'mean')]
                
                df_acc_all = pd.DataFrame(index=stats_dict['accuracy_by_all_uni'].index)
                df_acc_all['unigram'] = stats_dict['accuracy_by_all_uni'][('is_correct_uni', 'mean')]
                
                # end of initialisation
            
            df_acc_type[lambda_par] = stats_dict['accuracy_by_type_bi'][('is_correct_bi', 'mean')]
            df_acc_category[lambda_par] = stats_dict['accuracy_by_category_bi'][('is_correct_bi', 'mean')]
            df_acc_operation[lambda_par] = stats_dict['accuracy_by_operation_bi'][('is_correct_bi', 'mean')]
            df_acc_all[lambda_par] = stats_dict['accuracy_by_all_bi'][('is_correct_bi', 'mean')]
            
            lambda_col.append(lambda_par)
            accuracy_col.append(stats_dict['general_accuracy_bi'])
            diff_no_col.append(stats_dict['uni_bi_diff_no'])
            uni_better_bi.append(stats_dict['uni_better_bi_no'])
            bi_better_uni.append(stats_dict['bi_better_uni_no'])
            
            # save filtered dataframes (dictionary) to pickle
            with open('../results/pickles/lambda_{}_filtered_dfs.p'.format(lambda_par), 'wb') as file:
                filtered_dfs = get_filtered_dataframes(df_acc_all, df_answers)
                pickle.dump(filtered_dfs, file)
    
    # save dataframes with accuracy to excel
    with pd.ExcelWriter('../results/accuracy_by_stats.xlsx') as writer:
        df_acc_type.to_excel(writer, sheet_name='type')
        df_acc_category.to_excel(writer, sheet_name='category')
        df_acc_operation.to_excel(writer, sheet_name='operation')
        df_acc_all.to_excel(writer, sheet_name='all')
        
    # save lambda-accuracy, lambda-diff_no dataframe to excel
    lambda_accuracy_stats = pd.DataFrame(
    {
        'lambda': lambda_col,
        'accuracy': accuracy_col,
#         'total_diff': diff_no_col,
        'uni_better_bi': uni_better_bi,
        'bi_better_uni': bi_better_uni
    })
    
    # save single-files dict to excel
    single_values = pd.DataFrame(single_values_dict, index=range(len(single_values_dict.keys())))
    
    with pd.ExcelWriter('../results/general_stats.xlsx') as writer:
        lambda_accuracy_stats.to_excel(writer, sheet_name='by_lambda')
        single_values.to_excel(writer, sheet_name='single_values')
    
    print("Done all!")

In [137]:
wrapper_stats()

Analysing  test_set_with_answers_no_lang_error_scaling_0.20_lambda.csv

Different number of Nones in uni- and bigram model.
Uni - bi:  0
Bi - uni:  2
{'pct_nones': 0.06799417913993823}


/home/pawelhdd/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/pawelhdd/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Done!
Analysing  test_set_with_answers_no_lang_error_scaling_0.33_lambda.csv

Different number of Nones in uni- and bigram model.
Uni - bi:  0
Bi - uni:  2
{'pct_nones': 0.06799417913993823, 'unigram_acc': 0.8679200191189357}
Analysing  test_set_with_answers_no_lang_error_scaling_0.5_lambda.csv

Different number of Nones in uni- and bigram model.
Uni - bi:  0
Bi - uni:  2
{'pct_nones': 0.06799417913993823, 'unigram_acc': 0.8679200191189357}
Analysing  test_set_with_answers_no_lang_error_scaling_0.66_lambda.csv

Different number of Nones in uni- and bigram model.
Uni - bi:  0
Bi - uni:  2
{'pct_nones': 0.06799417913993823, 'unigram_acc': 0.8679200191189357}
Analysing  test_set_with_answers_no_lang_error_scaling_0.75_lambda.csv

Different number of Nones in uni- and bigram model.
Uni - bi:  0
Bi - uni:  2
{'pct_nones': 0.06799417913993823, 'unigram_acc': 0.8679200191189357}
Analysing  test_set_with_answers_no_lang_error_scaling_0.90_lambda.csv

Different number of Nones in uni- and bigra